In [209]:
import pickle
import ml.structures
import ml.dataset.corpus as corpus
import numpy as np

import pickle
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from keras.models import load_model

from keras.layers import LSTM
from keras.layers import Embedding
from keras.optimizers import RMSprop


from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.utils import np_utils
from keras.engine.topology import Input

from sklearn.metrics import accuracy_score
from keras.layers import concatenate, Merge
from keras.layers.merge import Concatenate

import keras
from keras.utils import to_categorical

from matplotlib import pyplot as plt
from IPython.display import clear_output

In [210]:
def make_16(chord):
    c = 16 / (128 / chord.duration)
    array = np.array([note.number for note in chord.notes])
    if len(array) == 0:
        array = [0]
    if c >= 1:
        return 1, np.tile(array, (int(c), 1))
    else:
        return 0, [0]

In [211]:
def cut_song(song, notes, chords):
    chords_track = []
    notes_track = np.array([])
    flag = 0
    for note in song.tracks[0].chords:
        if (make_16(note)[0] == 0):
            flag = 1
        notes_track = np.append(notes_track, make_16(note)[1])
    for chord in song.tracks[1].chords:
        if (make_16(chord)[0] == 0):
            flag = 1
        chords_track.append(make_16(chord)[1])
    if flag == 1:
        return 0, notes, chords
    return 1, notes_track, chords_track

In [212]:
def norm_array(X, size):
    for j in range(len(X)):
        X[j] = np.unique(np.array(X[j]))
        array = np.zeros(size-len(X[j])) - 1
        X[j] = np.append(X[j], array)
    X = np.array(X)
    return X

In [213]:
def create_X_Y(chords):
    real_chords = []
    y = []
    for i, subchord in enumerate(chords):
        if i != len(chords) - 1:
            for ssubchord in subchord:
                real_chords.append(ssubchord)
            size = len(subchord)
            for j in range(size):
                y.append(chords[i+1][-1])
    return real_chords, y

In [214]:
def notes_to_np(notes):
    for i in range(len(notes)):
        notes[i] = np.array(notes[i])
    notes = np.array(notes)

In [215]:
def create_unique_chords(X, unique_chords):
    for x in X:
        unique_chords = np.append(unique_chords, np.array(x))
    return unique_chords

In [216]:
def create_coded(X):
    return np.zeros(len(X))


In [217]:
def encode(X, coded_X):
    for j, k in enumerate(X):
        for i, unique_chord in enumerate(unique_chords):
            if np.array_equal(k, unique_chord):
                coded_X[j] = i

In [218]:
model_chords = Sequential()
model_notes = Sequential()

model_chords.add(Embedding(10000, 20, input_length=1, batch_input_shape=(1,1)))
model_notes.add(Embedding(128, 20, input_length=1, batch_input_shape=(1,1)))

merged_model = Sequential()
merged = Merge([model_chords, model_notes])
merged_model.add(merged)
merged_model.add(LSTM(20, stateful=True))
merged_model.add(Dense(10000, activation='softmax'))
merged_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


unique_chords = np.zeros(100)-1

with open('gtp_dataset_final.pickle', 'rb') as input_file:
    i = 0
    while True:
        try:
            i += 1
            song = ml.structures.Song()
            song.undump(input_file)
            chords = []
            notes = []
            flag_notes_chords = cut_song(song, notes, chords)
            if (flag_notes_chords[0] == 0) :
                continue
            notes = flag_notes_chords[1]
            chords = flag_notes_chords[2]
            X = []
            Y = []
            X, Y = create_X_Y(chords)
            X = norm_array(X, 100)
            Y = norm_array(Y, 100)
            notes_to_np(notes)
            unique_chords = create_unique_chords(X, unique_chords)
            unique_chords = create_unique_chords(Y, unique_chords)
            unique_chords = unique_chords.reshape((-1, 100))
            unique_chords = np.unique(unique_chords, axis=0)
            unique_chords = list(unique_chords)
            coded_X = []
            coded_Y = []
            coded_X = create_coded(X)
            coded_Y = create_coded(Y)
            encode(X, coded_X)
            encode(Y, coded_Y)
            X = np.array(coded_X)
            y = np.array(coded_Y)
            X_notes = notes
            X_notes = X_notes[:len(X)]
            X = X[:len(X_notes)]
            y = y[:len(X)]
            if (i < 100):
                merged_model.fit([X, X_notes], y, epochs=1, batch_size=1, verbose=1)
                merged_model.reset_states()
            else:
                break
                merged_model.predict([X, X_notes], batch_size=1, verbose=1)
            
            
        except EOFError:
            break

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


Epoch 1/1
446/446 [==============================] - 7s 16ms/step - loss: 4.5599 - acc: 0.1726
Epoch 1/1
286/286 [==============================] - 4s 14ms/step - loss: 2.5363 - acc: 0.1958
Epoch 1/1
90/90 [==============================] - 1s 12ms/step - loss: 2.7820 - acc: 0.3111
Epoch 1/1
90/90 [==============================] - 1s 12ms/step - loss: 2.4582 - acc: 0.3111
Epoch 1/1
94/94 [==============================] - 1s 12ms/step - loss: 9.6957 - acc: 0.0000e+00
Epoch 1/1
94/94 [==============================] - 1s 12ms/step - loss: 6.7173 - acc: 0.0106
Epoch 1/1
114/114 [==============================] - 1s 12ms/step - loss: 7.5716 - acc: 0.0000e+00
Epoch 1/1
94/94 [==============================] - 1s 12ms/step - loss: 5.9520 - acc: 0.0000e+00
Epoch 1/1
94/94 [==============================] - 1s 12ms/step - loss: 4.3923 - acc: 0.0745
Epoch 1/1
644/644 [==============================] - 8s 12ms/step - loss: 5.6053 - acc: 0.0311
Epoch 1/1
516/516 [==============================]

254/254 [==============================] - 3s 12ms/step - loss: 3.5381 - acc: 0.1772
Epoch 1/1
60/60 [==============================] - 1s 12ms/step - loss: 12.8267 - acc: 0.0000e+00
Epoch 1/1
274/274 [==============================] - 3s 12ms/step - loss: 9.6958 - acc: 0.0438
Epoch 1/1
240/240 [==============================] - 3s 12ms/step - loss: 8.1563 - acc: 0.0792
Epoch 1/1
116/116 [==============================] - 1s 12ms/step - loss: 4.2398 - acc: 0.1897
Epoch 1/1
302/302 [==============================] - 4s 13ms/step - loss: 2.4501 - acc: 0.3642
Epoch 1/1
260/260 [==============================] - 3s 11ms/step - loss: 1.9493 - acc: 0.2731
Epoch 1/1
134/134 [==============================] - 2s 11ms/step - loss: 10.8859 - acc: 0.0224
Epoch 1/1
126/126 [==============================] - 1s 11ms/step - loss: 7.7431 - acc: 0.0079
Epoch 1/1
254/254 [==============================] - 3s 11ms/step - loss: 7.6768 - acc: 0.0945


In [219]:
unique_chords

[array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1.]),
 array([ 0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -

In [220]:
import pandas as pd

df = pd.DataFrame(unique_chords)
df.to_csv("vocab.csv")
x = pd.read_csv("vocab.csv")
x = x.values.T[1:].T
x

array([[-1., -1., -1., ..., -1., -1., -1.],
       [ 0., -1., -1., ..., -1., -1., -1.],
       [24., 36., 43., ..., -1., -1., -1.],
       ...,
       [71., 83., -1., ..., -1., -1., -1.],
       [72., -1., -1., ..., -1., -1., -1.],
       [73., -1., -1., ..., -1., -1., -1.]])

In [221]:
merged_model.save("NN_model.h5")

In [222]:
model_chords = Sequential()
model_notes = Sequential()

model_chords.add(Embedding(10000, 20, input_length=1, batch_input_shape=(1,1)))
model_notes.add(Embedding(128, 20, input_length=1, batch_input_shape=(1,1)))

model = Sequential()
merged = Merge([model_chords, model_notes])
model.add(merged)
model.add(LSTM(20, stateful=True))
model.add(Dense(10000, activation='softmax'))

model.load_weights("NN_model.h5")
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


In [226]:
unique_chords[model.predict([np.array([0]), np.array([12.])], batch_size=1, verbose=1).argmax()]

1/1 [==============================] - 0s 3ms/step


array([ 0., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1.])